In [98]:
# Agrege acá el código para importar las librerias

# Es una libreria que da soporte para llamar funciones, clases y tipos del Lenguaje C
# Es un requerimiento para ts.flint
!pip3 install Cython

# Es una libreria especial para la serialización de arreglos basada en C++, permite la 
# integración entre pandas, NumPy, Spark ... 
# https://pypi.org/project/pyarrow/
# Es un requerimiento para ts.flint
!pip3 install pyarrow==0.9.0

# Es una colección de módulos para el análisis de series de tiempo con PySpark
# https://ts-flint.readthedocs.io/en/latest/
!pip3 install ts-flint

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Evaluacion iSofware, master=local[*]) created by __init__ at <ipython-input-1-9fb0c4765e98>:30 

# Integrantes del equipo
1. Catalina Gallego Yepes
2. Cesar Augusto Hernandez Rodriguez
3. Jessica Maria Rivera Jimenez
4. Jose Julian Mercado Garcia
5. Julio Cesar Vasquez Bedoya

In [110]:
!hdfs dfs -ls

Found 10 items
drwxr-xr-x   - root root       4096 2021-05-09 23:03 .ipynb_checkpoints
-rw-r--r--   1 root root      93121 2021-05-09 04:20 Asistencia.csv
-rw-r--r--   1 root root     110621 2021-05-09 03:40 Datos para el Trabajo.zip
-rw-r--r--   1 root root     111944 2021-05-09 04:21 Evaluacion.csv
-rw-r--r--   1 root root      87599 2021-05-09 20:38 Evaluacion_Pitch_limpio.csv
-rw-r--r--   1 root root      61180 2021-05-09 23:00 Taller (3).ipynb
-rw-r--r--   1 root root      33555 2021-05-09 22:04 Taller_Final.ipynb
-rw-r--r--   1 root root      38222 2021-05-10 00:42 Trabajo Final.ipynb
drwxr-xr-x   - root root       4096 2021-05-09 05:28 spark-warehouse
drwxr-xr-x   - root root       4096 2021-05-09 04:21 tmp


In [111]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

!hdfs dfs -copyFromLocal  Asistencia.csv /tmp/
!hdfs dfs -copyFromLocal  Evaluacion_Pitch_limpio.csv /tmp/

Asistencia = ContextoSql.read.load('/tmp/Asistencia.csv',
                                   format="csv",
                                   sep=',',
                                   inferSchema='true',
                                   header='true')
Evaluacion = ContextoSql.read.load('/tmp/Evaluacion_Pitch_limpio.csv',
                                   format="csv",
                                   sep=';',
                                   inferSchema='true',
                                   header='true')

copyFromLocal: `/tmp/Asistencia.csv': File exists
copyFromLocal: `/tmp/Evaluacion_Pitch_limpio.csv': File exists


In [112]:
df_Asistencia = Asistencia.withColumnRenamed("Marca temporal","Marca_temporal")\
                           .withColumnRenamed("Nombre de usuario","Nombre_de_usuario")\
                           .withColumnRenamed("Equipo al que perteneces:","Equipo_al_que_perteneces")\
                           .withColumnRenamed("Equipo que va a exponer:","Equipo_que_va_a_exponer")\

df_Asistencia.printSchema()



root
 |-- Marca_temporal: string (nullable = true)
 |-- Nombre_de_usuario: string (nullable = true)
 |-- Equipo_al_que_perteneces: string (nullable = true)
 |-- Equipo_que_va_a_exponer: string (nullable = true)



In [102]:
df_Evaluacion = Evaluacion.withColumnRenamed("Marca temporal","Marca_temporal")\
                        .withColumnRenamed("Nombre de usuario","Nombre_de_usuario")\
                        .withColumnRenamed("Equipo que vas a evaluar:","Equipo_a_evaluar")\
                        .withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","Introduccion")\
                        .withColumnRenamed("Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?","Equipo")\
                        .withColumnRenamed("Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?","Problema")\
                        .withColumnRenamed("Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?","Ventajas")\
                        .withColumnRenamed("Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?","Solucion")\
                        .withColumnRenamed("Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.","Producto")\
                        .withColumnRenamed("Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.","Traccion")\
                        .withColumnRenamed("Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.","Mercado")\
                        .withColumnRenamed("Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?","Competencia")\
                        .withColumnRenamed("Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ","Modelo_de_negocio")\
                        .withColumnRenamed("Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?","Inversion")\
                        .withColumnRenamed("Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.","Contacto")\
                        .withColumnRenamed("Exposición: ¿Qué tan coordinados estaban los expositores?","Exposicion1")\
                        .withColumnRenamed("Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?","Exposicion2")\
                        .withColumnRenamed("Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.","Exposicion3")\
                        .withColumnRenamed("Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)","supuesto_pagar")\
                        .withColumnRenamed("Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.","Observaciones")\

df_Evaluacion.printSchema()



root
 |-- Marca_temporal: string (nullable = true)
 |-- Nombre_de_usuario: string (nullable = true)
 |-- Equipo_a_evaluar: string (nullable = true)
 |-- Introduccion: integer (nullable = true)
 |-- Equipo: integer (nullable = true)
 |-- Problema: integer (nullable = true)
 |-- Ventajas: integer (nullable = true)
 |-- Solucion: integer (nullable = true)
 |-- Producto: integer (nullable = true)
 |-- Traccion: integer (nullable = true)
 |-- Mercado: integer (nullable = true)
 |-- Competencia: integer (nullable = true)
 |-- Modelo_de_negocio: integer (nullable = true)
 |-- Inversion: integer (nullable = true)
 |-- Contacto: integer (nullable = true)
 |-- Exposicion1: integer (nullable = true)
 |-- Exposicion2: integer (nullable = true)
 |-- Exposicion3: integer (nullable = true)
 |-- supuesto_pagar: string (nullable = true)
 |-- Observaciones: string (nullable = true)



In [103]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...

df_Asistencia.createOrReplaceTempView('Vista_Asistencia')

ContextoSql.sql("""
select distinct Nombre_de_usuario
from Vista_Asistencia
order by Nombre_de_usuario
""").show(10)


+--------------------+
|   Nombre_de_usuario|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
+--------------------+
only showing top 10 rows



In [104]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

df_Asistencia.createOrReplaceTempView('Vista_conteo')

ContextoSql.sql("""

SELECT Nombre_de_usuario,max(Marca_temporal) as max_Marca_temporal , min(Marca_temporal) as min_Marca_temporal , count(*)
from Vista_conteo
group by 1
""").show(30)

+--------------------+--------------------+--------------------+--------+
|   Nombre_de_usuario|  max_Marca_temporal|  min_Marca_temporal|count(1)|
+--------------------+--------------------+--------------------+--------+
|samolinap@unal.ed...|2021/04/13 9:37:2...|2021/04/08 8:11:4...|      10|
| jruedat@unal.edu.co|2021/04/13 9:41:2...|2021/04/08 8:12:5...|       8|
|nvalenciat@unal.e...|2021/04/13 9:39:3...|2021/04/08 8:10:1...|       8|
|serendona@unal.ed...|2021/04/13 9:40:2...|2021/04/08 8:10:4...|       8|
|jgutierrezlo@unal...|2021/04/13 9:39:0...|2021/04/08 8:12:0...|       8|
|dgerenal@unal.edu.co|2021/04/13 9:41:2...|2021/04/08 8:10:0...|       8|
|cjfunezg@unal.edu.co|2021/04/13 9:41:4...|2021/04/08 8:12:1...|       8|
|emflorezb@unal.ed...|2021/04/13 9:45:4...|2021/04/08 8:14:0...|       8|
|dchavarriar@unal....|2021/04/13 9:43:5...|2021/04/08 8:13:0...|       8|
|fguerrerot@unal.e...|2021/04/13 9:40:1...|2021/04/08 8:13:0...|       8|
|jdroldano@unal.ed...|2021/04/08 8:20:

In [105]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...


df_Asistencia.createOrReplaceTempView('Vista_conteo')

ContextoSql.sql("""

SELECT Nombre_de_usuario, count(*) as conteo
from Vista_conteo 
group by 1
having conteo>=1 and conteo<=2

""").show(30)

+--------------------+------+
|   Nombre_de_usuario|conteo|
+--------------------+------+
|jdroldano@unal.ed...|     1|
+--------------------+------+



In [16]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

revisar todos los que evaluaron y asistencias , left desde evaluaron + usuario asistencia , 

In [17]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

df_Asistencia.createOrReplaceTempView('Equipo_perteneces')

ContextoSql.sql("""
select distinct Nombre_de_usuario,Equipo_al_que_perteneces from Equipo_perteneces
order by 1 desc
""").show()

+--------------------+------------------------+
|   Nombre_de_usuario|Equipo_al_que_perteneces|
+--------------------+------------------------+
| wocampo@unal.edu.co|    Pertenezco a: Lo ...|
|   tleon@unal.edu.co|    Pertenezco a: Mi ...|
|serendona@unal.ed...|    Pertenezco a: Ges...|
|sednarvaezna@unal...|    Pertenezco a: Adm...|
|sebermudezg@unal....|    Pertenezco a: Ser...|
|seagudeloo@unal.e...|    Pertenezco a: Ges...|
|sasalazarr@unal.e...|    Pertenezco a: Te ...|
|samolinap@unal.ed...|    Pertenezco a: Adm...|
|saespinosab@unal....|    Pertenezco a: Adm...|
|sacastrot@unal.ed...|    Pertenezco a: Ser...|
|rmbuilesm@unal.ed...|    Pertenezco a: Adm...|
|parestrepoo@unal....|    Pertenezco a: Te ...|
|nvalenciat@unal.e...|    Pertenezco a: Te ...|
|nsalinas@unal.edu.co|    Pertenezco a: Adm...|
|mizapataa@unal.ed...|    Pertenezco a: Ges...|
|mimorenov@unal.ed...|    Pertenezco a: Te ...|
|mhernandezbe@unal...|    Pertenezco a: Te ...|
|mcarvajalsa@unal....|    Pertenezco a: 

In [116]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

df_Asistencia.createOrReplaceTempView('Equipo_perteneces')

ContextoSql.sql("""
select 
Equipo_que_va_a_exponer, 
count(Nombre_de_usuario) 
from Equipo_perteneces
where substring(Equipo_que_va_a_exponer,9,length(Equipo_que_va_a_exponer)) != substring(Equipo_al_que_perteneces,15,length(Equipo_al_que_perteneces))  --trim , upper 
group by 1 
ORDER BY 1 asc
""").show()

+-----------------------+------------------------+
|Equipo_que_va_a_exponer|count(Nombre_de_usuario)|
+-----------------------+------------------------+
|   Expone: Administr...|                      70|
|   Expone: Deserción...|                       3|
|   Expone: Gestión d...|                      72|
|   Expone: Gestión d...|                      72|
|   Expone: Gestión d...|                      74|
|   Expone: Lo tengo ...|                      75|
|   Expone: Mi Sanand...|                      72|
|   Expone: Servicios...|                      76|
|   Expone: Te lo cam...|                      75|
+-----------------------+------------------------+



In [106]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

df_Asistencia.createOrReplaceTempView('Equipo_perteneces')
df_Evaluacion.createOrReplaceTempView('Equipo_Evaluacion')

ContextoSql.sql("""
WITH BASE_ASISTENCIAS AS (
select 
Equipo_que_va_a_exponer, 
Equipo_al_que_perteneces,
Nombre_de_usuario,
substring(Equipo_al_que_perteneces,15,15) as llave_equipo_pertenece,
substring(Equipo_que_va_a_exponer,9,15) as llave_equipo_exponer
from Equipo_perteneces
),   BASE_EVALUACION AS (
select 
Nombre_de_usuario,
substring(Equipo_a_evaluar,1,15) as llave_Equipo_a_evaluar,
Equipo_a_evaluar
from Equipo_Evaluacion

)
SELECT  DISTINCT  t1.Nombre_de_usuario
FROM BASE_ASISTENCIAS t1 
inner join BASE_EVALUACION t2 on t1.Nombre_de_usuario=t2.Nombre_de_usuario  AND  T1.llave_equipo_pertenece=t2.llave_Equipo_a_evaluar
--GROUP BY 1,2,3
""").show()


+--------------------+
|   Nombre_de_usuario|
+--------------------+
| jruedat@unal.edu.co|
| kmolano@unal.edu.co|
|mcarvajalsa@unal....|
|daestradam@unal.e...|
|jhcordoba@unal.ed...|
|  lmayar@unal.edu.co|
|hriveraa@unal.edu.co|
|anapariciom@unal....|
+--------------------+



In [121]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...


ContextoSql.sql("""

WITH 
BASE_ASISTENCIAS AS (
select 
Equipo_que_va_a_exponer, 
Equipo_al_que_perteneces,
Nombre_de_usuario,
substring(Equipo_al_que_perteneces,15,15) as llave_equipo_pertenece,
substring(Equipo_que_va_a_exponer,9,15) as llave_equipo_exponer
from Equipo_perteneces
),   BASE_EVALUACION AS (
select *,
substring(Equipo_a_evaluar,1,15) as llave_Equipo_a_evaluar

from Equipo_Evaluacion
),

ASISTENTES_REALES AS (
SELECT 
t1.Nombre_de_usuario,
T1.Equipo_al_que_perteneces,
T2.*

FROM BASE_ASISTENCIAS t1 
inner join BASE_EVALUACION t2 on t1.Nombre_de_usuario=t2.Nombre_de_usuario
WHERE T1.llave_equipo_pertenece !=t2.llave_Equipo_a_evaluar 
)

    select Equipo_a_evaluar,
    ROUND(AVG(Introduccion),2) as a, 
    ROUND(AVG(Equipo),2) as b, 
    ROUND(AVG(Problema),2) as c, 
    ROUND(AVG(ventajas),2) as d, 
    ROUND(AVG(Solucion),2) as e,
    ROUND(AVG(Producto),2) as f,
    ROUND(AVG(Traccion),2) as g,
    ROUND(AVG(Mercado),2) as h,
    ROUND(AVG(Competencia),2) as i,
    ROUND(AVG(Modelo_de_negocio),2) as j,
    ROUND(AVG(inversion),2) as k,
    ROUND(AVG(contacto),2) as l,
    ROUND(AVG(Exposicion1),2) as m, 
    ROUND(AVG(Exposicion2),2) as n, 
    ROUND(AVG(Exposicion3),2) as o,
    ROUND(AVG(Introduccion+Equipo+Problema+ventajas+Solucion+Producto+Traccion+Mercado+Competencia+Modelo_de_negocio+inversion+contacto+Exposicion1+Exposicion2+Exposicion3)/15,2) AS PROMEDIO_TOTAL

    from ASISTENTES_REALES
    group by Equipo_a_evaluar
    order by Equipo_a_evaluar

""").show()


+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------------+
|    Equipo_a_evaluar|   a|   b|   c|   d|   e|   f|   g|   h|   i|   j|   k|   l|   m|   n|   o|PROMEDIO_TOTAL|
+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------------+
|Administración de...|4.11|4.42|3.97|3.91|4.17|4.44|4.03|4.43|3.93|4.42|4.11|4.14|4.31|4.22|4.13|          4.18|
|Deserción de estu...| 4.5| 5.0| 5.0| 5.0| 4.5| 5.0| 5.0| 4.5| 5.0| 4.5| 4.5| 5.0| 4.5| 4.5| 5.0|          4.77|
|Gestión de Menús ...|4.33|3.98|4.56|4.31|4.38|4.01|4.19|4.21|4.29|4.63| 4.3| 4.5|4.47|4.44|4.73|          4.36|
|Gestión de Reside...|4.55|4.45|4.64|4.46|4.45|4.12|4.53|4.71|4.41|4.52|4.34|4.66|4.42|4.51|4.42|          4.48|
|Gestión de progra...|4.38| 4.4|4.55|3.92|4.34|4.45|4.42| 4.5|4.28|4.32|4.54|4.43|4.45|4.49|4.63|          4.41|
|Lo tengo y te lo ...|4.49| 4.6|4.27|4.43|4.42|4.72|4.36|4.25|4.49|4.47|4.36|4.59|4.53| 4.6|4.61

In [140]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...


ContextoSql.sql("""


WITH 
BASE_ASISTENCIAS AS (
select 
Equipo_que_va_a_exponer, 
Equipo_al_que_perteneces,
Nombre_de_usuario,
substring(Equipo_al_que_perteneces,15,15) as llave_equipo_pertenece,
substring(Equipo_que_va_a_exponer,9,15) as llave_equipo_exponer
from Equipo_perteneces
),   BASE_EVALUACION AS (
select *,
substring(Equipo_a_evaluar,1,15) as llave_Equipo_a_evaluar

from Equipo_Evaluacion
),

ASISTENTES_REALES AS (
SELECT 
t1.Nombre_de_usuario,
T1.Equipo_al_que_perteneces,
T2.*

FROM BASE_ASISTENCIAS t1 
inner join BASE_EVALUACION t2 on t1.Nombre_de_usuario=t2.Nombre_de_usuario
WHERE T1.llave_equipo_pertenece !=t2.llave_Equipo_a_evaluar 
), final as (

    select Equipo_a_evaluar,
    ROUND(AVG(Introduccion),2) as a, 
    ROUND(AVG(Equipo),2) as b, 
    ROUND(AVG(Problema),2) as c, 
    ROUND(AVG(ventajas),2) as d, 
    ROUND(AVG(Solucion),2) as e,
    ROUND(AVG(Producto),2) as f,
    ROUND(AVG(Traccion),2) as g,
    ROUND(AVG(Mercado),2) as h,
    ROUND(AVG(Competencia),2) as i,
    ROUND(AVG(Modelo_de_negocio),2) as j,
    ROUND(AVG(inversion),2) as k,
    ROUND(AVG(contacto),2) as l,
    ROUND(AVG(Exposicion1),2) as m, 
    ROUND(AVG(Exposicion2),2) as n, 
    ROUND(AVG(Exposicion3),2) as o,
    ROUND(AVG(Introduccion+Equipo+Problema+ventajas+Solucion+Producto+Traccion+Mercado+Competencia+Modelo_de_negocio+inversion+contacto+Exposicion1+Exposicion2+Exposicion3)/15,2) AS PROMEDIO_TOTAL

    from ASISTENTES_REALES
    group by Equipo_a_evaluar
    order by Equipo_a_evaluar
)

select Equipo_a_evaluar, 'a' as item ,a as nota
from final inner join
(select max(a) nota
from final)  y on final.a=y.nota
union all
select Equipo_a_evaluar, 'b' as item  ,b as nota
from final inner join
(select max(b) nota
from final)  y on final.b=y.nota

union all
select Equipo_a_evaluar, 'c' as item  ,c as nota
from final inner join
(select max(c) nota
from final)  y on final.c=y.nota

union all
select Equipo_a_evaluar, 'd' as item  ,d as nota
from final inner join
(select max(d) nota
from final)  y on final.d=y.nota
union all
select Equipo_a_evaluar, 'e' as item  ,e as nota
from final inner join
(select max(e) nota
from final)  y on final.e=y.nota
union all
select Equipo_a_evaluar, 'f' as item  ,f as nota
from final inner join
(select max(f) nota
from final)  y on final.f=y.nota
union all
select Equipo_a_evaluar, 'g' as item  ,g as nota
from final inner join
(select max(g) nota
from final)  y on final.g=y.nota
union all
select Equipo_a_evaluar, 'h' as item  ,h as nota
from final inner join
(select max(h) nota
from final)  y on final.h=y.nota
union all
select Equipo_a_evaluar, 'i' as item  ,i as nota
from final inner join
(select max(i) nota
from final)  y on final.i=y.nota
union all
select Equipo_a_evaluar, 'j' as item  ,j as nota
from final inner join
(select max(j) nota
from final)  y on final.j=y.nota


""").show()


+--------------------+----+----+
|    Equipo_a_evaluar|item|nota|
+--------------------+----+----+
|Gestión de Reside...|   a|4.55|
|Deserción de estu...|   b| 5.0|
|Deserción de estu...|   c| 5.0|
|Deserción de estu...|   d| 5.0|
|Deserción de estu...|   e| 4.5|
|Servicios a domic...|   e| 4.5|
|Deserción de estu...|   f| 5.0|
|Deserción de estu...|   g| 5.0|
|Gestión de Reside...|   h|4.71|
|Deserción de estu...|   i| 5.0|
|Gestión de Menús ...|   j|4.63|
+--------------------+----+----+



In [ ]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...



In [ ]:

# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...
